In [21]:
%load_ext autoreload
%autoreload 2
import sys
import os
import time
import types
sys.path.append('../')
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive
from traitlets import traitlets
import matplotlib.pyplot as plt
from fbprophet import Prophet
import yaml
from src.data_downloader import DATA_REPOS, download_from_repo, get_dataframes
import plotly.graph_objects as go
from plotly.graph_objs import Layout
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, clear_output
with open("columns_names.yaml", 'r') as stream:
    out = yaml.load(stream)
    orig_data_columns = out['LABELS']['orig_data_columns']
    trend_labels = out['LABELS']['trend_labels']
import warnings
warnings.filterwarnings('ignore')
dest='../data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# first, set the main page as a Tab container
main_tab = widgets.Tab()
main_tab.children=[]

In [3]:
# first tab is for data download

main_tab.set_title(0, 'Italy')
download_tab = widgets.VBox()

download_out = widgets.Output(layout={'border': '1px solid black'})

@download_out.on_displayed
def check_creation_date(b=None):
    file1 = 'dpc-covid19-ita-regioni.csv'
    file2 = 'time_series_covid19_confirmed_global.csv'
    file_path1 = os.path.join(dest,file1)
    file_path2 = os.path.join(dest,file2)
    (mode, ino, dev, nlink, uid, gid, size, atime, mtime1, ctime) = os.stat(file_path1)
    (mode, ino, dev, nlink, uid, gid, size, atime, mtime2, ctime) = os.stat(file_path2)
    download_out.append_display_data("{} last modified: {}".format(file1, time.ctime(mtime1)))
    download_out.append_display_data("{} last modified: {}".format(file2, time.ctime(mtime2)))
    

update_button = widgets.Button(description='Update data from repos')
@update_button.on_click
def update_repo(b=None):
    download_out.append_display_data('downloading Italian data')
    download_from_repo(DATA_REPOS['italy']['url'], filenames=DATA_REPOS['italy']['streams'], dest=dest)
    download_out.append_display_data('downloading world data')
    download_from_repo(DATA_REPOS['world']['url'], filenames=DATA_REPOS['world']['streams'], dest=dest)  
    df_naz, reg, prov, df_reg, df_prov, df_world_confirmed, df_world_deaths, df_world_recovered, populations, ita_populations, df_comuni_sett = get_dataframes(dest, npt_rth=5, smooth=True)


download_tab.children = [widgets.HBox(children=[download_out, update_button])]

In [4]:
with download_out:
    df_naz, reg, prov, df_reg, df_prov, df_world_confirmed, df_world_deaths, df_world_recovered, populations, ita_populations, df_comuni_sett = get_dataframes(dest, npt_rth=5, smooth=True)
    

In [5]:
# second tab for italy
main_tab.set_title(1, "Download")
italy_tab = widgets.Tab()

In [111]:
prov_tab = widgets.VBox()
italy_tab.set_title(0, 'provinces')

province_out = widgets.Output(layout = {
            'width': '100%',
            'height': '900px',
            'border': '1px solid black'
        })
prov_data_columns = ['totale_casi', 'daily_totale_casi', '%daily_totale_casi', 'growth_factor']

label=widgets.Select(options=prov_data_columns)
top_prov=widgets.IntSlider(min=1,max=50,step=1,value=20)
date=widgets.DatePicker(description='Pick a Date',value=pd.to_datetime(df_prov.index.max()))
show_map=widgets.Checkbox(value=True)
show_grid=widgets.Checkbox(value=True)

def get_top_provinces(label, top_prov,date, show_map, show_grid):    
    with province_out:
        clear_output()
        label = [label]
        df_prov.index = pd.to_datetime(df_prov.index)
        tempdf = df_prov.loc[str(date)][['sigla_provincia','denominazione_provincia', 'lat', 'long']+ label].sort_values(by=label, 
             ascending=False)[:top_prov].set_index('sigla_provincia')

        if show_map:
            fig = px.choropleth(tempdf, 
                                geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson', 
                                locations='denominazione_provincia', 
                                color=label[0], 
                                color_continuous_scale='Reds',
                                featureidkey='properties.prov_name',                                   
                                range_color=(0, max(tempdf[label[0]])))
            # fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
            fig.update_layout(showlegend=True,title='italian provinces',
                              paper_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightgray'),plot_bgcolor='rgba(0,0,0,0)')
            fig.update_geos(resolution=50, showcountries=True, countrycolor="lightgray", showland=True, 
                            landcolor="Black", showsubunits=True, subunitcolor="Blue", showocean=True, oceancolor="Black",)
            fig.update_geos(fitbounds="locations", visible=False)
            fig.show()            
#             fig = px.scatter_mapbox(tempdf, 
#                         lat='lat', lon='long', color=label[0], size=label[0],
#                         labels = label[0],
#                         hover_name='denominazione_provincia',
#                         zoom=4.5,  height=800,
#                         mapbox_style="open-street-map",                        
#                         title='top {} provinces on day {} for {}'.format(top_prov, date.strftime("%m/%d/%Y"), label[0]),
#                )
#             fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightgray'))
#             fig.show()
        else:
            fig = px.bar(tempdf[label].reset_index(), x=label[0], y='sigla_provincia', orientation='h')
            fig.update_layout(showlegend=True,title='top {} provinces on day {}'.format(top_prov, date.strftime("%m/%d/%Y")),
                             paper_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightgray'),plot_bgcolor='rgba(0,0,0,0)')
            fig.update_xaxes(showgrid=show_grid, gridwidth=1, gridcolor='gray')
            fig.update_yaxes(showgrid=show_grid, gridwidth=1, gridcolor='gray')

            fig.show()

        
prov_widget = interactive(get_top_provinces,label=widgets.Select(options=prov_data_columns),
                            top_prov=widgets.IntSlider(min=1,max=50,step=1,value=20),
                            date=widgets.DatePicker(description='Pick a Date',value=pd.to_datetime(df_prov.index.max())),
                            show_map=widgets.Checkbox(value=True),
                            show_grid=widgets.Checkbox(value=True),
                            )
prov_box = widgets.HBox(prov_widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
prov_tab.children = [prov_box, province_out]

In [107]:
regions_tab = widgets.VBox()
italy_tab.set_title(1, 'forecast')
forecast_out = widgets.Output(layout = {
            'width': '100%',
            'height': '600px',
            'border': '1px solid black'
        })

def get_forecast(region,start_fit,end_fit,label,forecast_periods,smoothing):
    with forecast_out:
        clear_output()
        df = df_reg[region][label].rolling(smoothing).mean()
        y = label
        train_data = pd.DataFrame()
        train_data['ds']=pd.to_datetime(df.index)
        train_data['y']=np.log1p(df.reset_index(drop=True).values)
        train_data['floor'] = 0.
        m = Prophet(growth='linear', daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=False)
        m.fit(train_data.set_index('ds').loc[start_fit:end_fit].reset_index())
        future = m.make_future_dataframe(periods=forecast_periods)
        future['floor'] = train_data['floor']
        forecast = m.predict(future)
        forecast['yhat'] = np.expm1(forecast['yhat'])
        forecast['yhat_lower'] = np.expm1(forecast['yhat_lower'])
        forecast['yhat_upper'] = np.expm1(forecast['yhat_upper'])
        train_data['y'] = np.expm1(train_data['y'])
        df = pd.merge(left=train_data, right=forecast, on='ds', how='outer').set_index('ds')
        df.index = pd.to_datetime(df.index)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df['yhat_lower'],fill=None,mode='lines',line_color='lightgrey',name='confidence_lev_down'))
        fig.add_trace(go.Scatter(x=df.index, y=df['yhat_upper'],fill='tonexty',mode='lines',line_color='lightgrey', name='confidence_lev_up'))
        fig.add_traces(go.Scatter(x=df.index, y=df['y'], name='{}'.format(y), mode='lines+markers', marker=dict(size=5)))
        fig.add_traces(go.Scatter(x=df.loc[start_fit:end_fit].index, y=df['yhat'].loc[start_fit:end_fit], line_color='goldenrod',
                                  mode='lines',name='model fit'))
        fig.add_traces(go.Scatter(x=df.loc[end_fit:].index, y=df['yhat'].loc[end_fit:], line_color='darkblue', mode='markers',
                                  marker=dict(size=2), name='forecast'))
        fig.update_layout(title={'text':label.replace('_', ' ') + ' for ' + region, 'xanchor': 'left'})
        fig.update_layout(showlegend=True,paper_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightgray'),
                          plot_bgcolor='rgba(0,0,0,0)')
        fig.update_xaxes(showgrid=False, gridwidth=1, gridcolor='gray')
        fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='gray')
        fig.show()
        


forecast_widget=interactive(get_forecast,region=widgets.Dropdown(options=df_reg.keys(), value='Italy'), 
              start_fit=widgets.DatePicker(value=pd.to_datetime(df_naz.index[0])), 
              end_fit=widgets.DatePicker(value=pd.to_datetime(df_naz.index[-1])), 
              label = widgets.Dropdown(options=trend_labels, value='nuovi_positivi'),
              forecast_periods=50, smoothing=widgets.IntText(1))
forecast_box = widgets.HBox(forecast_widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))

regions_tab.children = [forecast_box, forecast_out]

In [112]:
italy_tab.children = [prov_tab, regions_tab]

In [113]:
# add all components to main_tab page
main_tab.children=[italy_tab, download_tab]

In [114]:
main_tab